# Rebin FITS images 2 x 2

For example, grizli processed JWST images at 20mas and 40mas / pixel

WHL0137 + Sunrise Arc + Earendel:  
https://s3.amazonaws.com/grizli-v2/JwstMosaics/whl0137/arc-v2/index.html

In [1]:
import os
from glob import glob
import numpy as np
from astropy.io import fits
import astropy.wcs as wcs

In [2]:
# Integer scaling of arrays. For non-integer scaling, see ndimage.map_coordinates or:
# from reproject import reproject_interp  # https://reproject.readthedocs.io/en/stable/

def magnify(a, n):
    """MAGNIFIES A MATRIX BY n
    YIELDING, FOR EXAMPLE:
    >>> a = np.arange(9).reshape(3, 3)
    >>> AA = magnify(a, 2)
    >>> AA
    001122
    001122
    334455
    334455
    667788
    667788
    """
    ny, nx = a.shape
    a = np.repeat(a, n**2)
    a = np.reshape(a, (ny,nx,n,n))
    a = np.transpose(a, (0, 2, 1, 3))
    a = np.reshape(a, (n*ny, n*nx))
    return a

def reshape_block(a, n):
    """(ny, nx) -> (n*n, ny/n, nx/n)"""
    ny, nx = np.array(a.shape) // n
    a = a[:ny*n,:nx*n]  # Trim if not even multiples
    a = np.reshape(a, (ny, n, nx, n))
    a = np.transpose(a, (0, 2, 1, 3))
    a = np.reshape(a, (ny, nx, n*n))
    a = np.transpose(a, (2, 0, 1))
    return a

def demagnify(a, n, func=np.mean):  # rebin
    """DEMAGNIFIES A MATRIX BY n
    YIELDING, FOR EXAMPLE:
    >>> demagnify(AA, 2)
    012
    345
    678
    """
    a = reshape_block(a, n)
    a = func(a, axis=0)
    return a

# Demonstrate
if 1:
    a = np.arange(9).reshape(3, 3)
    print(a, '<-- a')
    AA = magnify(a, 2)
    print(AA, '<-- AA')
    b = demagnify(AA, 2)
    print(b, '<-- b')

[[0 1 2]
 [3 4 5]
 [6 7 8]] <-- a
[[0 0 1 1 2 2]
 [0 0 1 1 2 2]
 [3 3 4 4 5 5]
 [3 3 4 4 5 5]
 [6 6 7 7 8 8]
 [6 6 7 7 8 8]] <-- AA
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]] <-- b


In [19]:
def report_pixel_scale(image_file):
    hdu = fits.open(image_file)
    filt = extract_filter(image_file)
    imwcs = wcs.WCS(hdu[idata].header, hdu)
    data = hdu[idata].data
    ny, nx = data.shape
    # image_pixel_scale = np.abs(hdu[0].header['CD1_1']) * 3600
    image_pixel_scale = wcs.utils.proj_plane_pixel_scales(imwcs)[0] 
    image_pixel_scale *= imwcs.wcs.cunit[0].to('arcsec')
    outline = filt.ljust(6)
    outline += ' %5d x %5d pixels' % (ny, nx)
    outline += ' = %6.2f" x %6.2f"' % (ny * image_pixel_scale, nx * image_pixel_scale)
    outline += ' (%.2f" / pixel)' % image_pixel_scale
    print(outline)

In [3]:
def extract_filter(image_file):
    return os.path.basename(image_file).split('_')[0].lower().split('-')[2]
    #return image_file.split('_')[0].lower().split('-')[1]

In [4]:
#image_files_list = glob('../images/*_i2d.fits')
#image_files_list = glob('../images/grizli_v2/20mas/*_sci.fits*')
image_files_list = glob('../images/*_sci.fits*')
image_files_list = list(np.sort(image_files_list))
#image_files_list = image_files_list[-1:] + image_files_list[:-1]  # move F770W first
image_files_list

['../images/nep-2mass-f070w-clear_drc_sci.fits',
 '../images/nep-2mass-f115w-clear_drc_sci.fits',
 '../images/nep-2mass-f200w-clear_drc_sci.fits',
 '../images/nep-2mass-f212n-clear_drc_sci.fits',
 '../images/nep-2mass-f277w-clear_drc_sci.fits',
 '../images/nep-2mass-f356w-clear_drc_sci.fits',
 '../images/nep-2mass-f410m-clear_drc_sci.fits',
 '../images/nep-2mass-f444w-clear_drc_sci.fits']

In [5]:
filters = list(map(extract_filter, image_files_list))

image_files_dict = {}
for i, filt in enumerate(filters):
    image_files_dict[filt] = image_files_list[i]
    print(filt, image_files_dict[filt])

f070w ../images/nep-2mass-f070w-clear_drc_sci.fits
f115w ../images/nep-2mass-f115w-clear_drc_sci.fits
f200w ../images/nep-2mass-f200w-clear_drc_sci.fits
f212n ../images/nep-2mass-f212n-clear_drc_sci.fits
f277w ../images/nep-2mass-f277w-clear_drc_sci.fits
f356w ../images/nep-2mass-f356w-clear_drc_sci.fits
f410m ../images/nep-2mass-f410m-clear_drc_sci.fits
f444w ../images/nep-2mass-f444w-clear_drc_sci.fits


In [6]:
#field = os.path.basename(image_files_list[0]).split('-')[0]
field = 'nep-2mass'
field

'nep-2mass'

In [7]:
#idata = 'sci'  # index where science data is
idata = 0  # index where science data is

In [8]:
# Check size of every image; they need to be the same, all pixel aligned

for filt in filters:
    report_pixel_scale(image_files_dict[filt])

f070w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f115w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f200w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f212n  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f277w  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
f356w  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
f410m  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
f444w  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)


In [9]:
# Remap all images to this pixel scale and use this image header

reference_filter = 'f200w'
reference_file = image_files_dict[reference_filter]
reference_hdu = fits.open(reference_file)
reference_header = reference_hdu[idata].header
reference_file

'../images/nep-2mass-f200w-clear_drc_sci.fits'

In [10]:
reference_hdu[idata].data.shape

(20000, 11500)

In [11]:
reference_header[:10]

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                11500                                                  
NAXIS2  =                20000                                                  
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =               5751.5 / Pixel coordinate of reference point            
CRPIX2  =              10001.5 / Pixel coordinate of reference point            
CD1_1   = -5.5555555555555E-06 / Coordinate transformation matrix element       
CD2_2   =  5.5555555555555E-06 / Coordinate transformation matrix element       

In [12]:
magnification_factor = 2
output_extension = '_20mas_sci'
output_directory = '../images/20mas'

In [24]:
for filt in 'f277w f356w f410m f444w'.split():
    image_file = image_files_dict[filt]
    print('LOADING', image_file)
    data = fits.getdata(image_file)
    #print(data.shape)
    report_pixel_scale(image_file)
    #print()
    #
    reprojected_file = image_file.replace('_drc_sci', output_extension)
    reprojected_file = os.path.basename(reprojected_file)
    reprojected_file = os.path.join(output_directory, reprojected_file)
    print('CREATING', reprojected_file)
    reprojected_data = magnify(data, magnification_factor)
    #print(reprojected_data.shape)
    fits.writeto(reprojected_file, reprojected_data, reference_header)
    report_pixel_scale(reprojected_file)
    print()

LOADING ../images/nep-2mass-f277w-clear_drc_sci.fits
f277w  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
CREATING ../images/20mas/nep-2mass-f277w-clear_20mas_sci.fits
f277w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)

LOADING ../images/nep-2mass-f356w-clear_drc_sci.fits
f356w  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
CREATING ../images/20mas/nep-2mass-f356w-clear_20mas_sci.fits
f356w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)

LOADING ../images/nep-2mass-f410m-clear_drc_sci.fits
f410m  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
CREATING ../images/20mas/nep-2mass-f410m-clear_20mas_sci.fits
f410m  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)

LOADING ../images/nep-2mass-f444w-clear_drc_sci.fits
f444w  10000 x  5750 pixels = 400.00" x 230.00" (0.04" / pixel)
CREATING ../images/20mas/nep-2mass-f444w-clear_20mas_sci.fits
f444w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)



In [25]:
#image_files_list = glob('../images/*_i2d.fits')
#image_files_list = glob('../images/grizli_v2/20mas/*_sci.fits*')
#image_files_list = glob('../images/*_sci.fits*')
image_files_list = glob('../images/20mas/*_sci.fits*')
image_files_list = list(np.sort(image_files_list))
#image_files_list = image_files_list[-1:] + image_files_list[:-1]  # move F770W first
image_files_list

['../images/20mas/nep-2mass-f070w-clear_drc_sci.fits',
 '../images/20mas/nep-2mass-f115w-clear_drc_sci.fits',
 '../images/20mas/nep-2mass-f200w-clear_drc_sci.fits',
 '../images/20mas/nep-2mass-f212n-clear_drc_sci.fits',
 '../images/20mas/nep-2mass-f277w-clear_20mas_sci.fits',
 '../images/20mas/nep-2mass-f356w-clear_20mas_sci.fits',
 '../images/20mas/nep-2mass-f410m-clear_20mas_sci.fits',
 '../images/20mas/nep-2mass-f444w-clear_20mas_sci.fits']

In [26]:
filters = list(map(extract_filter, image_files_list))

image_files_dict = {}
for i, filt in enumerate(filters):
    image_files_dict[filt] = image_files_list[i]
    print(filt, image_files_dict[filt])

f070w ../images/20mas/nep-2mass-f070w-clear_drc_sci.fits
f115w ../images/20mas/nep-2mass-f115w-clear_drc_sci.fits
f200w ../images/20mas/nep-2mass-f200w-clear_drc_sci.fits
f212n ../images/20mas/nep-2mass-f212n-clear_drc_sci.fits
f277w ../images/20mas/nep-2mass-f277w-clear_20mas_sci.fits
f356w ../images/20mas/nep-2mass-f356w-clear_20mas_sci.fits
f410m ../images/20mas/nep-2mass-f410m-clear_20mas_sci.fits
f444w ../images/20mas/nep-2mass-f444w-clear_20mas_sci.fits


In [27]:
# Check size of every image; they need to be the same, all pixel aligned

for filt in filters:
    report_pixel_scale(image_files_dict[filt])


f070w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f115w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f200w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f212n  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f277w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f356w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f410m  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
f444w  20000 x 11500 pixels = 400.00" x 230.00" (0.02" / pixel)
